In [1]:
from keras.layers import Input, Embedding, Dot, Reshape, Dense
from keras.models import Model
from keras.callbacks import LambdaCallback
import keras.backend as K
from keras.initializers import Constant, RandomUniform

Using TensorFlow backend.


In [2]:
import numpy as np
import random
random.seed(100)

# Where do Keras embedding values come from?
In this notebook I demonstrate what happens under the hood in Keras' Embedding Layer by examining a simple yet real-world model that generates book embeddings for a book recommendation system. The original article is described [here](https://towardsdatascience.com/neural-network-embeddings-explained-4d028e6f0526). Additionally, the author has also made his source code public [here](https://github.com/WillKoehrsen/wikipedia-data-science/blob/master/notebooks/Book%20Recommendation%20System.ipynb) and I'll draw heavily from it. I'll first give a high level overview of his approach but you should really go check out the source to get a fuller understanding! Then we'll peel back the veil and see how we can predict with certainty exactly how the embeddings are made. 

## Overview
The goal of the original project was to represent all the books on Wikipedia as vectors to create a book recommendation system. The supervised learning task requires the model to predict whether a link to a Wikipedia article appears in the article for a book. The hypothesis is that books with Wikipedia articles containing the same link to another Wikipedia article will themselves be similar. For example, the Wiki article [War and Peace](https://en.wikipedia.org/wiki/War_and_Peace) contains a link to [Anna Karenina](https://en.wikipedia.org/wiki/Anna_Karenina), itself a book. Thus these two books are likely related (indeed, they are written by the same author, [Leo Tolstoy](https://en.wikipedia.org/wiki/Leo_Tolstoy)). 

The model learns embedding vectors for each book such that similar books are closer to each other in the embedding space. Training the model consists of feeding it (book, link) pairs with a mix of positive (true) and negative (false) examples. For instance, a positive training example would be (War and Peace, Anna Karenina), while a negative one might be (War and Peace, kittens).  This practice of *negative sampling* is very common for training embeddings; the technique is also used in the word2vec and fastText word embeddings. 

## Book Embedding Model
The book embedding model is created with Keras and consists of 5 key components.  

1.  __Input Layer__: Each book is assigned a unique integer (*War and Peace* might be Book 3). Links are also assigned unique integers (*kittens* might be Link 0). A pair of integers representing a book and a link, (3, 0), is passed to the model. 
2.  __Embedding Layer__: These integers act as look-up values and select the corresponding row in the associated embedding matrix. 3 selects the third row vector in the book embedding matrix, while 0 selects the zeroth row in the link embedding matrix. 
<img src="book_embedding_matrix.png" width="400" height="400">

3. __Dot Product__: Both the book embedding vector and link embedding vector have the same dimension (the embedding dimension, which must be chosen beforehand). Our loss function requires a scalar so we take a dot product of the two vectors. 
4. __Loss Function__: The loss function is the Mean Squared Error which takes in the true label (in this case, -1 because the *kittens* link does NOT appear in the *War and Peace* Wiki article), and the value of the Dot Product produced in Step 3. 
5. __Optimization__: In this example we'll use the simplest algorithm, vanilla Stochastic Gradient Descent.

<img src="book_embedding_graph.png" width="500" height="500">


## Okay, so tell me where all these embedding numbers come from (aka, math time!)
The Keras Embedding Layer acts almost like a regular Dense Layer with two key differences. First, the Embedding Layer takes integers as input instead of real values. Secondly, the Embedding Layer typically does not have an activation function applied to it. But just like the weights in a Dense Layer, the embedding *weights* are updated during training through [backpropagation](https://en.wikipedia.org/wiki/Backpropagation). 

The weight update rules goes as

\begin{equation*}
w_{\text{book}_j}^{\text{(new)}} = w_{\text{book}_j}^{\text{(old)}} - \eta\delta_{\text{book}_j}
\end{equation*}

where each new element, *j*, of the book embedding vector can be computed by subtracting from the original value the quantity $\eta*\delta$. Here, $\eta$ is the learning rate and $\delta$ is the derivative of the cost function with respect to the weights of the book embedding. 

\begin{equation*}
\delta_{\text{book}_j} = \frac{\partial J}{\partial w_{\text{book}_j}} 
\end{equation*}


As discussed above, we know the cost function is the Mean Squared Error: 

\begin{equation*}
J = \frac{1}{n}\sum (Y_i - \hat Y_i)^2
\end{equation*}

In the example code that follows we'll be using a batch size of 1 so our cost function reduces to 

\begin{equation*}
J = (Y_i - \hat Y_i)^2
\end{equation*}

where $Y_i$ is the true label and $\hat Y_i$ is given by the dot product between the book embedding vector and link embedding vector:

\begin{equation*}
\hat Y_i = \mathbf{w}_{\text{book}} \cdot \mathbf{w}_{\text{link}} = \sum_{j=1}^d w_{\text{book}_j} w_{\text{link}_j}
\end{equation*}


Now we can use the chain rule to get
\begin{align}
\delta_{\text{book}_j} & = \frac{\partial J}{\partial w_{\text{book}_j}}\\ 
                       & = \frac{\partial J}{\partial \hat Y_i} \frac{\partial \hat Y_i}{\partial w_{\text{book}_j}}\\
                       & = 2(Y_i - \hat Y_i)w_{\text{link}_j}
\end{align}

Therefore, our final update rule is

\begin{equation*}
w_{\text{book}_j}^{\text{(new)}} = w_{\text{book}_j}^{\text{(old)}} - 2\eta(Y_i - \hat Y_i)w_{\text{link}_j}
\end{equation*}

We can perform the same analysis for the link embedding matrix and we'd get

\begin{equation*}
w_{\text{link}_j}^{\text{(new)}} = w_{\text{link}_j}^{\text{(old)}} - 2\eta(Y_i - \hat Y_i)w_{\text{book}_j}
\end{equation*}


So there we have it! If we did our math right, we should be able to use these to compute the new embedding values during training! Let's check our work with a real(ish) example.


## Let's see this in action!
Time to apply what we've learned to real code! I'm going to simplify the problem even further by pretending there are only 5 books on Wikipedia and only 3 links among them (what a sad universe). The original application had 50 embedding dimensions but I'm only going to use 5 so that the matrices we print out have a chance at being readable. Finally, I'm only going to feed in positive training examples, but the option exists for you to add in negative ones. I'll create a toy training set and we'll feed in one instance at a time. We'll print out the resulting embedding matrix after each batch and see how the embedding matrix updates. Then we'll code up the update rules we derived above and compare our results with Keras! 

## Create a fake training set

In [41]:
# In my sad universe there are only 5 books in the world and 3 links on the internet
num_books = 5
num_links = 3

# Each book is represented with integers 0 through 4. 
# Each link is represented with integers 0 through 2.
# The pair (0, 1) thus denotes (Book 0, Link 1)

# Now to create the training set itself: (book, link) pairs
# (I just made up these pairs but they'll do for our purpose)
pairs = [(0, 1), (1, 0), (2, 0), (3, 2), (4, 1), (0, 0), (1, 2), (2, 2)]

pairs_set = set(pairs)

## Create a batch generator
The *pairs* list above represents our entire (fake) training set. Each tuple corresponds to a (book, link) pair. We will train a Keras model with this data but we need to control how these examples are fed into the model so that we can understand the output. For that, we need a generator which I've based off the one in the original code [here](https://github.com/WillKoehrsen/wikipedia-data-science/blob/master/notebooks/Book%20Recommendation%20System.ipynb). This generator essentially loops through the training set in order, passing the requested number of training examples to the model. It also prints out which (book, link) pairs were sent, making it easy for us to keep track of the training process!

In [31]:
def generate_batch_in_order(pairs, n_positive=50, negative_ratio=1.0):
    """  
    This generator creates batches that are drawn in order from the input, pairs, which contains positive examples.
    Negative examples can also be generated according to the negative_ratio. 
    Once the generator has reached the end of the input, it loops back to the beginning. 
    
    This is not how you should train a NN in practice! But it is quite handy for examining the 
    output of the Keras model since we know exactly which examples it will see and in what order. 
    
    pairs          = list of (book integer, link integer) positive instances
    n_positive     = number of positive instances to include in the batch
    negative_ratio = ratio of positive to negative samples to include in the batch
    """
    
    batch_size = n_positive * (1 + negative_ratio)
    print("batch_size:", batch_size)
    batch = np.zeros((batch_size, 3))

    neg_label = -1

    j = 0
    i = 0
    while True:
        book_id, link_id = pairs[j]
        batch[i, :] = (book_id, link_id, 1)
        j += 1
        i += 1

        # If we reach the end of the number of "fixed" values for this array, send the batch
        if i >= n_positive:
            # But first, fill up the rest of the space with randomly drawn values
            for k in range(i, batch_size):
                # random selection
                random_book = random.randrange(books)
                random_link = random.randrange(links)

                # Check to make sure this is not a positive example
                if (random_book, random_link) not in pairs_set:
                    # Add to batch and increment index
                    batch[k, :] = (random_book, random_link, neg_label)

            print('\nbooks:', batch[:, 0], 'links:', batch[:, 1], 'labels:', batch[:, 2])
            yield {'book': batch[:, 0], 'link': batch[:, 1]}, batch[:, 2]
            i = 0

        # If we reach the end of the original array, reset indices
        if j == len(pairs):
            j = 0

## Create the Model
This model is pulled directly from the original [source code](https://github.com/WillKoehrsen/wikipedia-data-science/blob/master/notebooks/Book%20Recommendation%20System.ipynb) so go check it out! I've made some minor changes that streamline or simplify the model for the purpose of this tutorial and these are noted in the comments. One major addition is the use of Keras [Callbacks](https://keras.io/callbacks/), functions that can be applied at given stages of the training procedure. These allow us to print out the value of the embedding matrices after each batch. 

In [42]:
def book_embedding_model(embedding_size=5, book_callback=True, link_callback=False):
    """Model to embed books and wikilinks using the functional API.
       Trained to discern if a link is present in a article"""

    # Both inputs are 1-dimensional
    book = Input(name='book', shape=[1])
    link = Input(name='link', shape=[1])

    # Embedding the book (shape will be (None, 1, embedding_size))
    # Initial embedding values are seeded for reproducibility
    book_embedding = Embedding(name = 'book_embedding',
                               embeddings_initializer = RandomUniform(minval=-0.05, maxval=0.05, seed=42),
                               input_dim = num_books,
                               output_dim = embedding_size,
                               input_length=1)(book)

    # Embedding the link (shape will be (None, 1, embedding_size))
    # Initial embedding values are seeded for reproducibility
    link_embedding = Embedding(name = 'link_embedding',
                               embeddings_initializer = RandomUniform(minval=-0.05, maxval=0.05, seed=43),
                               input_dim = num_links,
                               output_dim = embedding_size)(link)

    # Merge the layers with a dot product along the second axis (shape will be (None, 1, 1))
    # Turned 'norm' off to simplify backprop math
    merged = Dot(name='dot_product', normalize=False, axes=2)([book_embedding, link_embedding])

    # Reshape to be a single number (shape will be (None, 1))
    merged = Reshape(target_shape=[1])(merged)

    model = Model(inputs=[book, link], outputs=merged)

    # Adding Callbacks to print out embedding matrices during training
    callbacks = []
    if book_callback:
        book_callback = LambdaCallback(
                on_batch_end=lambda batch, logs: print(model.get_layer('book_embedding').get_weights()[0]))
        callbacks.append(book_callback)
    
    if link_callback:
        link_callback = LambdaCallback(
                on_batch_end=lambda batch, logs: print(model.get_layer('link_embedding').get_weights()[0]))
        callbacks.append(link_callback)
        
    # Switching to SGD optimizer for simplification
    model.compile(optimizer='sgd', loss='mse')

    # Print out the initial embedding weights -- before any training has occured
    book_weights = model.get_layer("book_embedding").get_weights()[0]
    link_weights = model.get_layer("link_embedding").get_weights()[0]
    print("before-training book embedding matrix:\n", book_weights)
    print()
    print("before-training link embedding matrix:\n",link_weights)
    print()

    return model, callbacks

## Initialize the Model
I've added code above such that, upon model initialization, the book and link embedding matrices are printed out before training even occurs! I've seeded the random mechanism so that they are initialized to the same set of values every time (yay for reproducibility!). This is our embedding starting point and we'll use these to compare between Keras' update mechanism and our own. 

In [43]:
# Set the embedding dimension to be something that we can manageably print out
embedding_size = 5

# Batch size is one with no negative samples for illustrative purposes
n_positive = 1

# Create a training data generator
gen = generate_batch_in_order(pairs, n_positive, negative_ratio = 0)

# Instantiate model and show parameters
model, callbacks = book_embedding_model(embedding_size=embedding_size, book_callback=True, link_callback=False)
#print(model.summary())

before-training book embedding matrix:
 [[ 0.04522714  0.01774077  0.02953183  0.02557817 -0.00240444]
 [ 0.01310148 -0.03139796 -0.03856922 -0.01637782  0.02233351]
 [-0.02808004  0.03573376  0.03239204  0.00954127 -0.04970373]
 [-0.02527453  0.00060741 -0.01384113 -0.04551616  0.04721661]
 [ 0.03283885 -0.0085416   0.0101666  -0.01604132  0.00731027]]

before-training link embedding matrix:
 [[-0.02405849 -0.04089488 -0.043049   -0.04481744 -0.00090784]
 [-0.04059631 -0.00314282 -0.02813883 -0.00767363 -0.01062437]
 [ 0.00983997  0.0189384  -0.03238429 -0.01509678  0.04887613]]



We see that the book embedding matrix is randomly initialized and has shape (5,5) where each row corresponds to one of the 5 books in our universe and the columns represent the number of embedding dimensions. Similarly, the link embedding matrix has shape (3,5) because the embedding dimension is the same but there are only 3 lonely links in this sad universe. 

## Train the Model
Now let's see what happens when we train the model! The generator will pass ONE training example to the model at a time and the book embedding matrix will print out after each batch. 

In [36]:
model.fit_generator(gen, epochs = 1,
                    steps_per_epoch = len(pairs_set) // n_positive,
                    verbose = 2,
                    callbacks = callbacks,
                    workers=0)

Epoch 1/1
batch_size: 1

books: [0.] links: [1.] labels: [1.]
[[ 0.04441287  0.01767774  0.02896742  0.02542426 -0.00261754]
 [ 0.01310148 -0.03139796 -0.03856922 -0.01637782  0.02233351]
 [-0.02808004  0.03573376  0.03239204  0.00954127 -0.04970373]
 [-0.02527453  0.00060741 -0.01384113 -0.04551616  0.04721661]
 [ 0.03283885 -0.0085416   0.0101666  -0.01604132  0.00731027]]

books: [1.] links: [0.] labels: [1.]
[[ 0.04441287  0.01767774  0.02896742  0.02542426 -0.00261754]
 [ 0.01262192 -0.03221313 -0.03942733 -0.01727117  0.02231541]
 [-0.02808004  0.03573376  0.03239204  0.00954127 -0.04970373]
 [-0.02527453  0.00060741 -0.01384113 -0.04551616  0.04721661]
 [ 0.03283885 -0.0085416   0.0101666  -0.01604132  0.00731027]]

books: [2.] links: [0.] labels: [1.]
[[ 0.04441287  0.01767774  0.02896742  0.02542426 -0.00261754]
 [ 0.01262192 -0.03221313 -0.03942733 -0.01727117  0.02231541]
 [-0.02855724  0.03490115  0.03151336  0.00863601 -0.049713  ]
 [-0.02527453  0.00060741 -0.01384113 -0.

## Examine the training output
Compare the before-training book embedding matrix to how it looks after processing the first training example: (0, 1). 

\begin{align}
\text{before-training} & = 
    \begin{bmatrix}
        0.04522714 & 0.01774077 & 0.02953183 & 0.02557817 & -0.00240444\\
        0.01310148 & -0.03139796 & -0.03856922 & -0.01637782 & 0.02233351\\
        -0.02808004 & 0.03573376 & 0.03239204 & 0.00954127 & -0.04970373\\
        -0.02527453 & 0.00060741 & -0.01384113 & -0.04551616 & 0.04721661\\
        0.03283885 & -0.0085416 &  0.0101666 & -0.01604132 & 0.00731027
    \end{bmatrix} \\
    \\
\text{after-one-example} & = 
    \begin{bmatrix}
        \mathbf{0.04441287} & \mathbf{0.01767774} & \mathbf{0.02896742} & \mathbf{0.02542426} & \mathbf{-0.00261754}\\
        0.01310148 & -0.03139796 & -0.03856922 & -0.01637782 & 0.02233351 \\
        -0.02808004 & 0.03573376 & 0.03239204 & 0.00954127 & -0.04970373 \\
        -0.02527453 & 0.00060741 & -0.01384113 & -0.04551616 & 0.04721661\\
        0.03283885 & -0.0085416 &  0.0101666 & -0.01604132 & 0.00731027
    \end{bmatrix}
\end{align}

If you look closely you'll see that only the 0th vector has new values in it! All the other vectors are the same as before! This is exactly what we expected from our analysis above. If you follow the training output you'll see that each row of the book embedding matrix only receives updated values after the model processes a training example containing the corresponding book integer. 

Now that we know which values are updating, let's verify that we understand exactly how we get the new values. For this, we'll create our own update rules according to the backpropagation math we did earlier. 

## Create our own update rules
The math we did earlier comes in handy now! We'll use it to create functions that compute the new embedding matrix. We'll need to pass both the book and link embeddings to both functions because they are dependent on each other (thanks to the dot product), as well as the learning rate, $\eta$, and the true label, $Y_i$. 

\begin{align}
w_{\text{book}_j}^{\text{(new)}} & = w_{\text{book}_j}^{\text{(old)}} - 2 \eta (Y_i - \hat Y_i)w_{\text{link}_j}\\
\\
w_{\text{link}_j}^{\text{(new)}} & = w_{\text{link}_j}^{\text{(old)}} - 2\eta(Y_i - \hat Y_i)w_{\text{book}_j}
\end{align}

In [21]:
# Functions for updating embedding weights for the Book Embedding model according to the rules of backpropagation.

def update_book_embedding(book_embedding, link_embedding, label, learning_rate):
    # Compute the dot product, y_hat
    y_hat = np.dot(book_embedding, link_embedding)
    
    # Compute the updated weights according to:
    # w_book_new = w_book_old - learning_rate * delta_book
    #            = w_book_old - learning_rate * (2 * (label - y_hat) * link_embedding)
    
    book_embedding_new = book_embedding - learning_rate * 2 * (y_hat - label) * link_embedding
    
    return book_embedding_new


def update_link_embedding(book_embedding, link_embedding, label, learning_rate):
    # Compute the dot product
    y_hat = np.dot(book_embedding, link_embedding)
    
    # Compute the updated weights according to
    # w_link_new = w_link_old - learning_rate * delta_link
    #            = w_link_old - learning_rate * (2 * (label - y_hat) * book_embedding)
    
    link_embedding_new = link_embedding - learning_rate * 2 * (y_hat - label) * book_embedding
    
    return link_embedding_new

## Test our update rules
We'll start with the same before-training embedding matrices that the Keras model started with. Then we'll simply loop through the training set (pairs) and apply our update rules. If we did the math right, the resulting output should match what Keras output above. 

In [45]:
# The initial book and link embedding matrices before any training 
# (Go ahead -- check that these are the same as above)

book_embedding = np.array([[ 0.04522714,  0.01774077,  0.02953183,  0.02557817, -0.00240444],
                           [ 0.01310148, -0.03139796, -0.03856922, -0.01637782,  0.02233351],
                           [-0.02808004,  0.03573376,  0.03239204,  0.00954127, -0.04970373],
                           [-0.02527453,  0.00060741, -0.01384113, -0.04551616,  0.04721661],
                           [ 0.03283885, -0.0085416 ,  0.0101666 , -0.01604132,  0.00731027]])

link_embedding = np.array([[-0.02405849, -0.04089488, -0.043049,   -0.04481744, -0.00090784],
                           [-0.04059631, -0.00314282, -0.02813883, -0.00767363, -0.01062437],
                           [ 0.00983997,  0.0189384,  -0.03238429, -0.01509678,  0.04887613]])

In [46]:
for p in pairs:

    # the first value in a pair is an integer representing a book
    book_idx = p[0]
    # the second value in a pair is an integer representing a link
    link_idx = p[1]
    print("book:", book_idx, "link:", link_idx)

    # since we are looping only through positive examples, the true label is 1 for each of them
    # The default learning rate for SGD in Keras is 0.01
    book_embedding[book_idx] = update_book_embedding(book_embedding[book_idx], link_embedding[link_idx], 1, 0.01)
    link_embedding[link_idx] = update_link_embedding(book_embedding[book_idx], link_embedding[link_idx], 1, 0.01)

    print("New book embedding:")
    print(book_embedding)
    #print("new link embedding:")
    #print(link_embedding)
    print()

book: 0 link: 1
New book embedding:
[[ 0.04441286  0.01767773  0.02896742  0.02542425 -0.00261754]
 [ 0.01310148 -0.03139796 -0.03856922 -0.01637782  0.02233351]
 [-0.02808004  0.03573376  0.03239204  0.00954127 -0.04970373]
 [-0.02527453  0.00060741 -0.01384113 -0.04551616  0.04721661]
 [ 0.03283885 -0.0085416   0.0101666  -0.01604132  0.00731027]]

book: 1 link: 0
New book embedding:
[[ 0.04441286  0.01767773  0.02896742  0.02542425 -0.00261754]
 [ 0.01262192 -0.03221312 -0.03942732 -0.01727117  0.02231541]
 [-0.02808004  0.03573376  0.03239204  0.00954127 -0.04970373]
 [-0.02527453  0.00060741 -0.01384113 -0.04551616  0.04721661]
 [ 0.03283885 -0.0085416   0.0101666  -0.01604132  0.00731027]]

book: 2 link: 0
New book embedding:
[[ 0.04441286  0.01767773  0.02896742  0.02542425 -0.00261754]
 [ 0.01262192 -0.03221312 -0.03942732 -0.01727117  0.02231541]
 [-0.02855744  0.03490083  0.03151303  0.00863565 -0.04971302]
 [-0.02527453  0.00060741 -0.01384113 -0.04551616  0.04721661]
 [ 0.0

## Let's compare Keras' book embedding matrix after one training example to ours!
<br>

\begin{align}
\text{Keras} & = 
    \begin{bmatrix}
        \mathbf{0.04441287} & \mathbf{0.01767774} & \mathbf{0.02896742} & \mathbf{0.02542426} & \mathbf{-0.00261754}\\
        0.01310148 & -0.03139796 & -0.03856922 & -0.01637782 & 0.02233351 \\
        -0.02808004 & 0.03573376 & 0.03239204 & 0.00954127 & -0.04970373 \\
        -0.02527453 & 0.00060741 & -0.01384113 & -0.04551616 & 0.04721661\\
        0.03283885 & -0.0085416 &  0.0101666 & -0.01604132 & 0.00731027
    \end{bmatrix} \\
\\
\text{Ours} & =
    \begin{bmatrix}
        \mathbf{0.04441286} & \mathbf{0.01767773} & \mathbf{0.02896742} & \mathbf{0.02542425} & \mathbf{-0.00261754}\\
        0.01310148 & -0.03139796 &-0.03856922 &-0.01637782 & 0.02233351\\
        -0.02808004 & 0.03573376 & 0.03239204 & 0.00954127& -0.04970373\\
        -0.02527453 & 0.00060741 &-0.01384113 & -0.04551616 & 0.04721661\\
        0.03283885 &-0.0085416 &  0.0101666 & -0.01604132 & 0.00731027\\
    \end{bmatrix}
\end{align}

## We've matched Keras down to 7 decimal places!!!
Close enough for me to feel like we've got a good handle on how these embeddings are actually learned. 